# Databases for Developers - Assignment 2
The following tasks should be handed in on Peergrade.

## Task 1 - investigation
Produce a small writeup (around 300 words) answering the following questions.

#### 1. What is point of NoSQL databases?

Ulemper ved relationelle databaser:

- De er svære at skalere horisontalt til flere maskiner (fremfor at øge RAM eller CPU)
- De er komplekse at implementere
- De er ikke fleksible (skemaerne er ikke fleksible)
- De er ikke fejltolerante


NoSQL-databaser forsøger at imødekomme disse mangler, da de er:

- Skemaløs
- Distribueret
- Fejltolerant

#### 2. What is the CAP theorem?

CAP er teori om databaseopbygning og står for:

- Consistency: Hver bruger får den samme data fra alle kilder under alle omstændigheder. Hvis en node mister forbindelsen, stopper den med at give brugerne svar i stedet for at svare dem med information der måske er forkert, fordi informationen kan være ændret på en anden node.	


- Avalibility: Hvis noder i systemet mister forbindelsen kan systemet stadig anvendes. Vi acceptere f.eks. at noget data leveret til brugerne kan være forkert, til gengæld virker vores system oftere. 


- Partition tolerance: Hvis et databasesystem er fordelt over flere servere, kan systemets opbygning håndtere hvis forbindelsen mellem dele af systemet brydes. Altså går en server ned eller mister forbindelsen til resten af systemet, kan en bruger stadig få fat i indholdet via de andre serverere.

The CAP theorem siger, at du kan vælge to af disse egenskaber, og optimer til disse to når man opbygger databasesystemer


#### 3. What are ideal use cases of HBase?

Det vigtigste formål med brugen af en NoSQL-database er distribuerede datalagre ved behov for  datalagring af enorme mængder data. NoSQL bruges til bl.a. realtime web apps.

Apache HBase bruges til læse- / skriveadgang i realtime til Big Data. Det er en ikke-relationel database modelleret efter Googles Bigtable. 


## Task 2 - Bloom filters
Bloom filters are used in hbase as an incredible optimization. Solve the following.

#### 1. What is a bloom filter? 

Et Bloom-filter er en pladseffektiv datastruktur, der bruges til at teste, om et element er med i et sæt


#### 2. What is an advantage of bloom filters over hash tables?

Bloom filtre fylder meget mindre end et hash tables, derudover er søgning efter elementer meget hurtigere

#### 3. What is a disadvantage of bloom filters? 

Bloom flitre kan give false positive svar

#### 4. Using your language of choice, implement a bloom filter with add and check functions. The backing bit-array can simply be a long (64 bit integer).

In [1]:
# Python 3 program to build Bloom Filter
# Install mmh3 and bitarray 3rd party module:
# pip install mmh3
# pip install bitarray

import math
import mmh3
from bitarray import bitarray
import numpy as np
from collections import OrderedDict

In [2]:
class BloomFilter():

    def __init__(self):
        self.a = bitarray(2**6)
        self.a.setall(False)

    def add(self, item):
        for i in range(3):
            digest = mmh3.hash(item, i) % 64
            self.a[digest] = 1     

    def check(self, item):
        for i in range(3):
            digest = mmh3.hash(item, i) % 64
            if self.a[digest] == False:
                return False
        return True

In [3]:
bf = BloomFilter()

bf.add("Yarn")
bf.check("Plants")

False

In [4]:
bf.check("Yarn")

True

#### 5. If you are to store one million ASCII strings with an average size of 10 characters in a hash set, what would be the approximate space consumption?

Hver karakter fylder 1 byte derfor 10 * 1.000.000 = 10.000.000 bytes

1 byte = 8 bits
10.000.000 bytes * 8 = 80.000.000 bits

#### 6. The following equation gives the required number of bits of space per inserted key, where E is the false positive rate.

b = 1.44log2(1/E)

#### 7. How many bits per element are required for a 1% false positive rate?

1,44*log2(1/0,01) = 9,56 bits svarende til ca. 10 bits

#### 8. How many bits per element are required for a 5% false positive rate? 

1,44*log2(1/0,05) = 6,22 bits svarende til ca. 6 bits

#### 9. If you are to store one million ASCII strings with an average size of 10 characters in a bloom filter, what would be the approximate space consumption, given an allowed false positive rate of 5%?

6,22 * 1.000.000 = 6.220.000 bits

## Task 3 - Huffman coding

HBase internally uses a compression that is a combination of LZ77 and Huffman
Coding.

#### 1. Generate Huffmann Code (and draw the Huffmann Tree) based on the following string: “beebs beepps!!!!! their eerie ears hear pears”

In [41]:
# Huffman Coding in python

# Creating tree nodes
class NodeTree(object):

    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return (self.left, self.right)

    def nodes(self):
        return (self.left, self.right)

    def __str__(self):
        return '%s_%s' % (self.left, self.right)


# Main function implementing huffman coding
def huffman_code_tree(node, left=True, binString=''):
    if type(node) is str:
        return {node: binString}
    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, True, binString + '0'))
    d.update(huffman_code_tree(r, False, binString + '1'))
    return d

# Calculating frequency
def calc_char_frequency(input_str):
    chars = {}
    for c in input_str:
        if c in chars:
            chars[c] += 1
        else:
            chars[c] = 1

    chars = sorted(chars.items(), key=lambda x: x[1], reverse=True)
    return chars

def make_huffman_code(input_str):
    nodes = calc_char_frequency(input_str)
    while len(nodes) > 1:
        (key1, c1) = nodes[-1]
        (key2, c2) = nodes[-2]
        nodes = nodes[:-2]
        node = NodeTree(key1, key2)
        nodes.append((node, c1 + c2))

        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
    huffman_code = huffman_code_tree(nodes[0][0])
    return huffman_code  

In [46]:
huffman_code = make_huffman_code("beebs beepps!!!!! their eerie ears hear pears")

print(huffman_code)
print('----------------------')

print(' Char | Huffman code ')
print('----------------------')
for (char, frequency) in freq:
    print(' %-4r |%12s' % (char, huffmanCode[char]))

{'h': '0000', 't': '00010', 'i': '00011', 'r': '001', '!': '010', 'a': '0110', 'p': '0111', 'e': '10', ' ': '110', 'b': '1110', 's': '1111'}
----------------------
 Char | Huffman code 
----------------------
 'e'  |          10
 ' '  |         110
 '!'  |         010
 'r'  |         001
 's'  |        1111
 'b'  |        1110
 'p'  |        0111
 'a'  |        0110
 'h'  |        0000
 'i'  |       00011
 't'  |       00010


#### 2. How many bits is the compressed string? How many bits is the raw ASCII string?


In [68]:
def bits_size(input_str):
    num_of_bytes = len(input_str.encode('utf-8'))
    num_of_bits = num_of_bytes * 8 # 1 byte = 8 bits
    return num_of_bits

In [70]:
compressed_str = ""
    
for key in huffman_code.keys():
    compressed_str += huffman_code[key]


print("Compressed string:", bits_size(compressed_str))
print("Raw ASCII string:", bits_size("beebs beepps!!!!! their eerie ears hear pears"))

Compressed string: 328
Raw ASCII string: 360


#### 3. Compress “pete is here” with the Huffmann tree from before.

In [71]:
huffman_code_pete = make_huffman_code("pete is here")
print(huffman_code_pete)

{'e': '0', 'r': '1000', 'h': '1001', ' ': '101', 't': '1100', 'p': '1101', 's': '1110', 'i': '1111'}


## Task 4 - Map and Reduce

Solve the following using Javascript, for example in your browser’s developer
console.

#### 1. Map the list of numbers to a list of their square roots: [1, 9, 16, 100]

In [72]:
from math import sqrt 

nums = [1, 9, 16, 100] 
result = list(map(lambda x: sqrt(x), nums)) 
result

[1.0, 3.0, 4.0, 10.0]

#### 2. Map the list of words so each is wrapped in a  < h1 > tag: [“Intro”, “Requirements”, “Analysis”, “Implementation”, “Conclusion”, “Discussion”,“References”]

In [73]:
words = ['Intro', 'Requirements', 'Analysis', 'Implementation', 'Conclusion', 'Discussion', 'References']
result = list(map(lambda x: "<h1>{}</h1>".format(x), words))
result

['<h1>Intro</h1>',
 '<h1>Requirements</h1>',
 '<h1>Analysis</h1>',
 '<h1>Implementation</h1>',
 '<h1>Conclusion</h1>',
 '<h1>Discussion</h1>',
 '<h1>References</h1>']

#### 3. Use map to uppercase the words (all letters): [“i’m”, “yelling”, “today”]

In [74]:
words =  ["i’m", "yelling", "today"]

result = list(map(lambda x: x.upper(), words))
result

['I’M', 'YELLING', 'TODAY']

#### 4. Use map to transform words into their lengths: [“I”, “have”, “looooooong”,“words”]

In [75]:
words =   ["I", "have", "looooooong", "words"]

result = list(map(lambda x: len(x), words))
result

[1, 4, 10, 5]

#### 5. Get the json file comics.json from the course site. Paste it into your browser’s Javascript console. Use map to get all the image urls, and wrap them in img-tags.

In [80]:
comics = [
{
  "month": "1",
  "num": 43,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Red Spiders 2",
  "transcript": "[[Red spiders, with round appendages at the end of each of their six legs, are seen navigating an environment of blocks and other geometric constructions]]\n{{title text: This was actually drawn years before Red Spiders}}",
  "alt": "This was actually drawn years before Red Spiders",
  "img": "https://imgs.xkcd.com/comics/red_spiders_2.jpg",
  "title": "Red Spiders 2",
  "day": "1"
},
{
  "month": "1",
  "num": 44,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Love",
  "transcript": "[[A man and a woman stand facing one another]]\nMan: I love you!\nWoman: I love you!\nMan: I love you more!\nWoman: Yeah.\n[[A man and a woman stand facing one another - saying nothing.]]\n{{Alt-text: This one makes me wince every time I think about it}}",
  "alt": "This one makes me wince every time I think about it",
  "img": "https://imgs.xkcd.com/comics/love.jpg",
  "title": "Love",
  "day": "1"
},
{
  "month": "1",
  "num": 45,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Schrodinger",
  "transcript": "[[Label: Schrödinger's Comic]]\n[[Two figures standing, one with a black hat]]\nThe last panel of this comic is both funny and not funny at the same time.\nUntil you read it, there's no way to tell which it will end up being.\nShit.\n{{alt: There was no alt-text until you moused over}}",
  "alt": "There was no alt-text until you moused over",
  "img": "https://imgs.xkcd.com/comics/schrodinger.jpg",
  "title": "Schrodinger",
  "day": "4"
},
{
  "month": "1",
  "num": 46,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Secrets",
  "transcript": "I just want you to share in my secrets\n[[lonely looking girl staring down]]\nand not run away\n{{alt: I'm a big fan of Kurt Halsey}}",
  "alt": "I'm a big fan of Kurt Halsey",
  "img": "https://imgs.xkcd.com/comics/secrets.jpg",
  "title": "Secrets",
  "day": "6"
},
{
  "month": "1",
  "num": 47,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Counter-Red Spiders",
  "transcript": "[[A stack of stick figures, standing on each others shoulders extends from the bottom of the frame to the top.  Cuboids hang in the air]]\nThe counter-red-spider offensive begins ...\n{{title text: I hope we can stop them}}",
  "alt": "I hope we can stop them",
  "img": "https://imgs.xkcd.com/comics/counter-red-spiders.jpg",
  "title": "Counter-Red Spiders",
  "day": "9"
},
{
  "month": "1",
  "num": 48,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Found",
  "transcript": "[[A male and female stick figure are standing on a white hill (presumably snow) with a grey sky covered with thick streaks of white, and small pink dots]]\nwe are just two people \nwho found each other\n{{No more, no less}}",
  "alt": "No more, no less",
  "img": "https://imgs.xkcd.com/comics/found.jpg",
  "title": "Found",
  "day": "12"
},
{
  "month": "1",
  "num": 49,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Want",
  "transcript": "I want to be brave enough to tell you how I feel.\nI want to say \"I love you\" _before_ I hang up the phone for once.\nI want to drive all night with you, listening to mix tapes, not caring where we end up.\nOh, and I also really want to get with your sister.\nI mean, DAMN.\n{{title text: Well, she's pretty hot.}}",
  "alt": "Well, she's pretty hot.",
  "img": "https://imgs.xkcd.com/comics/want.jpg",
  "title": "Want",
  "day": "14"
},
{
  "month": "1",
  "num": 50,
  "link": "",
  "year": "2006",
  "news": "",
  "safe_title": "Penny Arcade",
  "transcript": "Tycho: You know what? If you've never played the 1995 SNES RPG 'Seiken Densetsu' don't even _bother_ reading today's strip. We don't _need_ your kind here.\n{{title text: Of course, Penny Arcade has already mocked themselves for this. They don't care.}}",
  "alt": "Of course, Penny Arcade has already mocked themselves for this.  They don't care.",
  "img": "https://imgs.xkcd.com/comics/penny_arcade.jpg",
  "title": "Penny Arcade",
  "day": "17"
}
]

In [81]:
result = list(map(lambda x: "<img>{}</img>".format(x["img"]), comics))
result

['<img>https://imgs.xkcd.com/comics/red_spiders_2.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/love.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/schrodinger.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/secrets.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/counter-red-spiders.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/found.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/want.jpg</img>',
 '<img>https://imgs.xkcd.com/comics/penny_arcade.jpg</img>']

#### 6. Use reduce to sum the array of numbers: [1,2,3,4,5]

In [83]:
import functools
import operator

numbers = [1,2,3,4,5]
print (functools.reduce(operator.add,numbers))
#Alternativly --> print (functools.reduce(lambda a,b : a+b,numbers))

15


#### 7. Use reduce to sum the x-value of the objects in the array: [{x: 1},{x: 2},{x: 3}]

In [85]:
x_values =  [{"x": 1},{"x": 2},{"x": 3}]

result = functools.reduce(lambda x, y: x + y['x'], x_values, 0)
result

6

#### 8. Use reduce to flatten an array of arrays: [[1,2],[3,4],[5,6]]

In [86]:
numbers_array =  [[1,2],[3,4],[5,6]]

result = functools.reduce(lambda x, y: x+y, numbers_array)
result

[1, 2, 3, 4, 5, 6]

#### 9. Use reduce to return an array of the positive numbers: [-3, -1, 2, 4, 5]

In [88]:
numbers = [-3, -1, 2, 4, 5]

result = list(filter(lambda x: x>0, numbers))
result

[2, 4, 5]